In [1]:
import math
import itertools
from queue import PriorityQueue
from collections import defaultdict, deque
from dataclasses import dataclass, field

In [2]:
# filename = "sample.txt"
filename = "sample2.txt"
# filename = "input.txt"
with open(filename, encoding="utf-8") as f:
    data = f.read()

lines = data.strip().split("\n")

https://adventofcode.com/2024/day/16

Very rough code today, but at least it works!

In [3]:
maze = set()
for y, line in enumerate(lines):
    for x, c in enumerate(line):
        pos = complex(x, y)
        match c:
            case "#":
                # A wall is functionally the same as a hole
                continue
            case "S":
                start_pos = pos
            case "E":
                end_pos = pos
        maze.add(pos)


In [4]:
def dist(p1: complex, p2: complex) -> float:
    d = p2 - p1
    return abs(d.real) + abs(d.imag)

def turn_dist(p1: complex, facing: complex, p2: complex) -> int:
    # Relative direction of end from start
    d = p2 - p1
    # For each axis, we're aligned if diff=0, or facing the right direction if sign(facing) matches sign(diff)
    # X
    if d.real == 0:
        x_turn = 0
    elif facing.real == 0:
        x_turn = 1
    elif math.copysign(1, d.real) == facing.real:
        x_turn = 0
    else:
        # Facing directly opposite, 2x 90deg turns
        x_turn = 2
    # Y
    if d.imag == 0:
        y_turn = 0
    elif facing.imag == 0:
        y_turn = 1
    elif math.copysign(1, d.imag) == facing.imag:
        y_turn = 0
    else:
        y_turn = 2

    return max(x_turn, y_turn)

def estimated_cost(pos: complex, facing: complex, end: complex) -> int:
    return int(dist(pos, end) + 1000 * turn_dist(pos, facing, end))

In [5]:
@dataclass(order=True)
class Cell:
    priority: int
    score: int=field(compare=False)
    pos: complex=field(compare=False)
    facing: complex=field(compare=False)
    path: list=field(default_factory=list, compare=False)

def turns(facing: complex) -> list[tuple[int, complex]]:
    return [(1, facing * -1j), (1, facing * 1j), (2, facing * -1)]

def next_steps(maze: set[complex], start: complex, facing: complex) -> list[tuple[int, complex]]:
    out = []
    for i in itertools.count(1):
        pos = start + facing * i
        if pos not in maze:
            break
        out.append((i, pos))
    return out

def a_star_bests(maze: set[complex], start: complex, facing: complex, end: complex):
    """
    Note: Works for Part 1, but fails for part 2. Two major issues:
    1. This prunes all alternate paths to intermediate nodes using the seen set
      (The final paths returned probably only differ by the final node the destination is approached from)
    2. Since this takes big leaps at every step, it doesn't get all intermediate nodes in the line
    """
    candidates = PriorityQueue()
    seen = set()
    best_score = None
    candidates.put(Cell(0, 0, start, facing, [(start, facing)]))
    while not candidates.empty():
        cell = candidates.get()
        if (cell.pos, cell.facing) in seen:
            continue
        
        # Part 2: Keep going until we've found ALL best paths
        if (best_score is not None) and (cell.score > best_score):
            return

        if cell.pos == end:
            if best_score is None:
                print(f"Reached goal in {len(cell.path)} moves, {cell.score=}")
                best_score = cell.score
            yield cell
            # Nowhere to go from end
            # Don't add end cell to seen set!
            continue

        seen.add((cell.pos, cell.facing))
        
        # Try cells which we can reach from this one
        # Turns
        for i, new_facing in turns(cell.facing):
            score = cell.score + 1000 * i
            est_dist = estimated_cost(cell.pos, new_facing, end)
            candidates.put(Cell(score + est_dist, score, cell.pos, new_facing, cell.path + [(cell.pos, new_facing)]))
        # Steps
        for i, new_pos in next_steps(maze, cell.pos, cell.facing):
            score = cell.score + i
            est_dist = estimated_cost(new_pos, cell.facing, end)
            candidates.put(Cell(score + est_dist, score, new_pos, cell.facing, cell.path + [(new_pos, cell.facing)]))
    print(f"A* couldn't find a path to {end}")



In [6]:
## Part 1
# Pathfinding through a maze! What's the cost of the shortest path?
# Step forwards = 1 point, turn 90 degrees = 1000 points
# Start on S, facing East. Reach E (any direction)
a_star_generator = a_star_bests(maze, start_pos, 1, end_pos)
result = next(a_star_generator)
result.score

Reached goal in 30 moves, cell.score=11048


11048

In [7]:
## Part 2
# How many tiles are part of at least one of the best paths through the maze?
# Get all best paths, then take unique cell positions from all paths
# best_results = [result]
# best_results.extend(a_star_generator)
# len(best_results)

In [8]:
## Attempt 2
# Take 1 step/turn at a time
# Track alternate min-length routes to intermediate nodes 
# Instead of tracking full route to each cell, just keep the set of predecessor nodes
# To trace the full path, repeatedly get the set of predecessors until we reach start
def find_bests(maze: set[complex], start: complex, facing: complex, end: complex):
    best_end = None
    best_scores = {(start, facing): 0}
    predecessors = defaultdict(list)

    candidates = PriorityQueue()
    candidates.put(Cell(0, 0, start, facing, [None, (start, facing)]))
    while not candidates.empty():
        cell = candidates.get()
        if (best_end is not None) and (cell.score > best_end):
            # Already exceeded max score
            break
        if (cell.pos, cell.facing) in best_scores:
            if cell.score > best_scores[(cell.pos, cell.facing)]:
                # Not as good as best so far for this cell
                continue
        
        best_scores[(cell.pos, cell.facing)] = cell.score
        predecessors[(cell.pos, cell.facing)].append(cell.path[-2])

        if cell.pos == end:
            print(f"Reached goal in {len(cell.path)} moves, {cell.score=}")
            # Nowhere to go from end
            best_end = cell.score
            continue
        
        # Try cells which we can reach from this one
        # Note: don't bother estimating distance remaining for part 2
        # Turns
        for turn in (-1j, 1j):
            new_facing = cell.facing * turn
            score = cell.score + 1000
            # est_dist = estimated_cost(cell.pos, new_facing, end)
            candidates.put(Cell(score, score, cell.pos, new_facing, cell.path + [(cell.pos, new_facing)]))
        # 1 step
        if (new_pos := (cell.pos + cell.facing)) in maze:
            score = cell.score + 1
            # est_dist = estimated_cost(new_pos, cell.facing, end)
            candidates.put(Cell(score, score, new_pos, cell.facing, cell.path + [(new_pos, cell.facing)]))
    
    # Done pathfinding. Return all predecessors of end
    predecessors[(start, facing)] = []
    return predecessors

def all_predecessors(predecessors, end: complex) -> set[complex]:
    seen = set()
    nodes = deque([(pos, facing) for (pos, facing) in predecessors.keys() if pos == end])
    while nodes:
        pos, facing = nodes.pop()
        if (pos, facing) in seen:
            continue
        seen.add((pos, facing))
        nodes.extend(predecessors.get((pos, facing), []))
    return seen

In [9]:
bests = find_bests(maze, start_pos, 1, end_pos)
visited_nodes = all_predecessors(bests, end_pos)
visited_positions = {pos for (pos, facing) in visited_nodes}
len(visited_positions)

Reached goal in 61 moves, cell.score=11048
Reached goal in 61 moves, cell.score=11048


64